In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd # Recommended for easier data handling after scraping
from time import sleep

In [2]:
# url = "https://results.finishtime.co.za/results.aspx?CId=35&RId=30205&EId=1&dt=0&PageNo=1"

In [3]:
def extract_table_data(table):
    # 2. Extract table headers from the first <tr>
    # The provided HTML indicates headers are in <td> tags within the first <tr>,
    # which has specific styling for headers.
    headers = []
    # Find the first <tr> in the table
    first_row = table.find('tr')

    if not first_row:
        print("Error: No rows found in the table, cannot extract headers.")
        return None

    # Extract text from all <td> tags in the first row for headers
    # We need to be careful about duplicate columns due to responsive design classes
    # 'd-none d-sm-table-cell' for desktop/larger screens
    # 'd-sm-none d-xs-table-cell' for mobile/smaller screens
    # Let's prioritize the desktop view headers if both exist, and filter out empty ones.
    potential_headers = first_row.find_all('td')
    
    # Manually define logic for header extraction based on content or specific attributes
    # We will iterate through all <td>s in the first row,
    # but try to extract meaningful text from their direct children (e.g., <a> tags).
    # We also need to manage duplicates like "Name" appearing twice.

    # Heuristic: If a <td> contains an <a> tag, use the text of the <a>.
    # Otherwise, use the direct text of the <td>.
    # We'll collect all potential headers and then clean them.
    for td in potential_headers:
        # Check for a specific structure (like <a> tag with text) for headers
        link = td.find('a', class_='ltw-nodecor')
        if link:
            header_text = link.get_text(strip=True)
        else:
            # For headers without <a>, like 'Fav', 'Share', 'Cat Pos', 'Gen Pos', 'Country'
            # get direct text, stripping whitespace and potentially extra content.
            header_text = td.get_text(strip=True)

        # Filter out social share icons that might mistakenly be picked up as headers
        # These are typically in 'd-none d-sm-table-cell' but are content, not headers.
        # We can also check if text is empty or contains non-alphabetic characters (like sort icons)
        if header_text and not (
            'fa-star' in str(td) or # for 'Fav' column icon in header td
            'x-twitter.svg' in str(td) or # for 'Share' column icons in header td
            'facebook.svg' in str(td) or
            'print.svg' in str(td) or
            'youtube.svg' in str(td)
        ):
            headers.append(header_text)
    
    # Remove potential duplicate "Name" header if both desktop and mobile versions are present
    # The mobile 'Name' header has d-sm-none d-xs-table-cell class
    # The desktop 'Name' header has d-none d-sm-table-cell class
    # Let's ensure we only take the one with 'width:100%' for the main Name column
    cleaned_headers = []
    seen_names = False
    for i, header in enumerate(headers):
        # Check the original td's class to determine which 'Name' it is
        original_td = potential_headers[i] # This mapping might be tricky due to filtering above
        
        # A more reliable way: define the expected headers manually, then map data.
        # However, for automated extraction, we try to parse them.

        # Simplified approach: if we see "Name" twice, keep the first one
        if header == 'Name':
            if not seen_names:
                cleaned_headers.append(header)
                seen_names = True
        elif header == '': # Skip empty headers which might come from image columns
            continue
        else:
            cleaned_headers.append(header)
    
    # Manual inspection of the provided HTML structure reveals the actual headers
    # are: Fav, Pos, Share, Race No, Wave, (Flag), Name, Time, Net Time, Category, Cat Pos, Gender, Gen Pos, Club, Country
    # The empty <td> in the headers is likely for the flag column.
    # Let's refine header extraction to be more robust for the given HTML structure:
    # The first <tr> has 16 <td> elements. We need to map them to meaningful headers.

    # Based on the HTML provided, the headers should be:
    # Fav, Pos, Share, Race No, Wave, (empty for flag), Name (desktop), Name (mobile), Time, Net Time, Category, Cat Pos, Gender, Gen Pos, Club, Country
    # We need to pick one 'Name' and identify the empty one for the flag.

    # Let's rebuild header extraction knowing the structure.
    headers = []
    for td in first_row.find_all('td'):
        # Text from <a> tag within <td> is usually the header, or direct text from <td>
        link = td.find('a', class_='ltw-nodecor')
        if link:
            headers.append(link.get_text(strip=True))
        else:
            # Handle cases like 'Fav', 'Share', 'Cat Pos', 'Gen Pos', 'Country' where text is direct
            # or where it's an image cell without text in the header.
            text = td.get_text(strip=True)
            if 'fa-star' in str(td) and 'Fav' not in headers: # Specifically for the 'Fav' header
                    headers.append('Fav')
            elif 'Share on X' in str(td) and 'Share' not in headers: # Specifically for the 'Share' header
                headers.append('Share')
            elif text: # If there's direct text, use it
                headers.append(text)
            else: # For the empty flag header or other complex hidden ones
                headers.append('') # Placeholder, will be cleaned

    # Post-process headers to fix duplicates and empty ones from responsive design
    final_headers = []
    seen_name = False
    for i, h in enumerate(headers):
        if h == 'Name':
            if not seen_name:
                final_headers.append('Name')
                seen_name = True
            # Skip the second 'Name' which is the mobile version
        elif h == '':
            # This could be the flag column. We'll name it 'Flag' or skip it if it's not a real column
            # Given the data structure, it's the 6th header (index 5)
            if i == 5: # The empty <td> that precedes the main 'Name' column usually for a flag
                final_headers.append('Flag')
            elif 'd-sm-none' in potential_headers[i].get('class', []): # This is likely the mobile pos column that duplicates desktop pos
                continue
            else:
                final_headers.append(h) # Keep other empty ones, might be visual spacers. We'll clean them later.
        elif h == 'Pos' and len(final_headers) > 0 and final_headers[-1] == 'Pos':
            # Skip duplicate 'Pos' from responsive design if it's already added
            continue
        else:
            final_headers.append(h)
    
    # Remove empty strings from headers and ensure uniqueness if necessary (though the HTML implies distinct columns)
    final_headers = [h for h in final_headers if h] # Filter out empty strings
    
    # If headers are still not as expected, manually force them or ensure unique names
    # Based on visual inspection of the table structure, a good set of headers would be:
    # ['Fav', 'Pos', 'Share', 'Race No', 'Wave', 'Flag', 'Name', 'Time', 'Net Time', 'Category', 'Cat Pos', 'Gender', 'Gen Pos', 'Club', 'Country']
    # The 'Share' column is actually an action column (social share icons), not a data point itself
    # The 'Fav' is also an action column
    # Let's adjust the `headers` list to reflect meaningful columns for the DataFrame
    
    # A more robust approach for headers:
    # Iterate through the first row's <td> elements, identify the intended column name
    # based on context (text or child <a> text)
    actual_headers = []
    # List of classes that indicate a desktop/larger screen cell to prioritize
    desktop_visible_classes = ['d-none', 'd-sm-table-cell']
    
    # Map original header positions to desired names based on your HTML
    # This is more robust than trying to parse dynamically for this complex header row.
    # This is a bit of a manual mapping given the mixed content and responsive classes.
    # I'll rely on the text of the `<a>` tag or the direct `<td>` text for now.
    
    # Let's try to infer headers based on the text within <a> tags and direct <td> text.
    # Some columns don't have sortable links.
    inferred_headers = []
    for i, td in enumerate(first_row.find_all('td')):
        link_text = ''
        if td.find('a', class_='ltw-nodecor'):
            link_text = td.find('a', class_='ltw-nodecor').get_text(strip=True)
        
        # Special handling for ambiguous/empty headers
        if i == 0: # Fav column
            inferred_headers.append('Fav_Action') # It's an action, not a data column
        elif i == 1: # Pos (Desktop)
            inferred_headers.append('Pos')
        elif i == 2: # Pos (Mobile) - skip this as it's a duplicate of Pos
            continue
        elif i == 3: # Share action column
            inferred_headers.append('Share_Action') # It's an action
        elif i == 4: # Race No
            inferred_headers.append('Race No')
        elif i == 5: # Wave
            inferred_headers.append('Wave')
        elif i == 6: # Flag Image
            inferred_headers.append('Flag')
        elif i == 7: # Name (Desktop)
            inferred_headers.append('Name')
        elif i == 8: # Name (Mobile) - this cell contains Name and Race No, Category for mobile
            # We've already captured 'Name' and 'Race No' from desktop.
            # If we need the combined 'Name #RaceNo Category' for mobile, it's a separate parsing task.
            # For a clean DataFrame, let's skip this as it duplicates 'Name' and 'Race No'.
            continue
        elif i == 9: # Time
            inferred_headers.append('Time')
        elif i == 10: # Net Time
            inferred_headers.append('Net Time')
        elif i == 11: # Category
            inferred_headers.append('Category')
        elif i == 12: # Cat Pos
            inferred_headers.append('Cat Pos')
        elif i == 13: # Gender
            inferred_headers.append('Gender')
        elif i == 14: # Gen Pos
            inferred_headers.append('Gen Pos')
        elif i == 15: # Club
            inferred_headers.append('Club')
        elif i == 16: # Country
            inferred_headers.append('Country')
        else:
            inferred_headers.append(link_text if link_text else td.get_text(strip=True))

    headers = [h for h in inferred_headers if h and h != 'i'] # Filter out empty and stray 'i' from font-awesome

    # Final check on headers, ensure consistency and no redundant columns
    # Based on your HTML, the most useful headers are:
    # Pos, Race No, Wave, Flag, Name, Time, Net Time, Category, Cat Pos, Gender, Gen Pos, Club, Country
    # The "Fav" and "Share" columns are action buttons, so we might want to skip them or
    # represent them as boolean/link columns if needed. For a clean data table,
    # often these action columns are excluded.
    # Let's adjust to extract data relevant to runner details.

    # Let's identify the actual data columns by their position or content.
    # The key is to map the data cells to the correct header.

    # Given the HTML structure, the header row contains TDs.
    # We need to map the data from the subsequent TDs to the correct header names.
    # Let's simplify the header extraction to focus on the text, and then manually
    # filter out columns we don't want (like the action buttons or duplicate mobile columns).

    # Re-attempting header extraction focusing on text content and filtering:
    headers_raw = [td.get_text(strip=True) for td in first_row.find_all('td')]
    
    # The header texts with their indices in the provided HTML:
    # 0: 'Fav' (from font awesome icon, but its <td> text is empty) - skip
    # 1: 'Pos' (desktop)
    # 2: 'Pos' (mobile) - skip
    # 3: '' (Share action column) - skip
    # 4: 'Race No'
    # 5: 'Wave'
    # 6: '' (Flag icon column) - use 'Flag'
    # 7: 'Name' (desktop)
    # 8: 'Name' (mobile, combined) - skip
    # 9: 'Time'
    # 10: 'Net Time'
    # 11: 'Category'
    # 12: 'Cat Pos'
    # 13: 'Gender'
    # 14: 'Gen Pos'
    # 15: 'Club'
    # 16: 'Country'
    
    # Define the target headers and their corresponding column indices in the HTML
    # This is the most reliable way given the complex HTML structure.
    header_map = {
        1: 'Pos',           # Pos (desktop)
        4: 'Race No',
        5: 'Wave',
        6: 'Flag',          # This column has an img, not text, so we'll treat it specially
        7: 'Name',          # Name (desktop)
        9: 'Time',
        10: 'Net Time',
        11: 'Category',
        12: 'Cat Pos',
        13: 'Gender',
        14: 'Gen Pos',
        15: 'Club',
        16: 'Country'
    }
    
    headers = [header_map[i] for i in sorted(header_map.keys())]
    print(f"Final Headers identified: {headers}")

    # Extract table rows (data)
    data_rows = []
    # Find all <tr> tags, starting from the second <tr> (index 1) as the first is headers
    all_tr_elements = table.find_all('tr')

    # Skip the first <tr> which contains headers
    for tr in all_tr_elements[1:]: # Start from the second row
        row_data = []
        all_tds_in_row = tr.find_all('td')

        # Extract data based on the header_map indices
        for i, td in enumerate(all_tds_in_row):
            if i in header_map:
                # Special handling for 'Flag' column: extract img src
                if header_map[i] == 'Flag':
                    img_tag = td.find('img')
                    if img_tag and 'src' in img_tag.attrs:
                        # Extract just the country code from the flag image URL if possible
                        # e.g., 'https://images.racetec.net/flags/16/ZA.png' -> 'ZA'
                        src = img_tag['src']
                        country_code = src.split('/')[-1].split('.')[0] if '/' in src else ''
                        row_data.append(country_code)
                    else:
                        row_data.append('') # No flag found
                # Special handling for 'Name' column (desktop version) and 'Category'
                # These might contain nested <a> tags whose text we want.
                elif header_map[i] in ['Name', 'Category', 'Gender', 'Club', 'Country']:
                    # Get text from immediate child <a> if available, otherwise direct text
                    link_tag = td.find('a', class_='ltw-name') 
                    if link_tag:
                        row_data.append(link_tag.get_text(strip=True))
                    else: # Fallback for country which might not have a link or direct text
                        row_data.append(td.get_text(strip=True))
                # For simple text columns, get direct text content
                else:
                    # For other columns, get text, stripping whitespace
                    row_data.append(td.get_text(strip=True))
        
        if row_data and len(row_data) == len(headers):  # Ensure row has correct number of columns
            data_rows.append(row_data)
        elif row_data:
                print(f"Warning: Skipping row due to column mismatch: {row_data} (Expected {len(headers)} columns, got {len(row_data)})")


    if data_rows:
        print(f"Successfully extracted {len(data_rows)} data rows.")
    else:
        print("No data rows found in the table body.")
        return None

    # 3. Create Pandas DataFrame
    print("Creating Pandas DataFrame...")
    df = pd.DataFrame(data_rows, columns=headers)
    print("DataFrame created with headers.")
    
    return df

In [4]:
all_data = pd.DataFrame()  # Initialize an empty DataFrame to collect all data
for page in range(1, 455):  # Adjust the range for more pages if needed
    sleep(3.57)  # Be polite to the server, avoid too many requests in a short time
    url = f"https://results.finishtime.co.za/results.aspx?CId=35&RId=30205&EId=1&dt=0&PageNo={page}"
    print(url)
    response = requests.get(url)
    response.raise_for_status()  # Raise an HTTPError for bad responses (4xx or 5xx)

    # 2. Parse the HTML
    soup = BeautifulSoup(response.text, 'html.parser')

    # Find the table containing the results
    table = soup.find('table', class_="table table-sm small align-middle ltw-cell-center")

    if not table:
        print(f"No results found on page {page}.")
        continue

    page_data = extract_table_data(table)

    if page_data is not None:
        all_data = pd.concat([all_data, page_data], ignore_index=True)
        print(f"Data from page {page} added. Total rows so far: {len(all_data)}")

https://results.finishtime.co.za/results.aspx?CId=35&RId=30205&EId=1&dt=0&PageNo=1
Final Headers identified: ['Pos', 'Race No', 'Wave', 'Flag', 'Name', 'Time', 'Net Time', 'Category', 'Cat Pos', 'Gender', 'Gen Pos', 'Club', 'Country']
Successfully extracted 50 data rows.
Creating Pandas DataFrame...
DataFrame created with headers.
Data from page 1 added. Total rows so far: 50
https://results.finishtime.co.za/results.aspx?CId=35&RId=30205&EId=1&dt=0&PageNo=2
Final Headers identified: ['Pos', 'Race No', 'Wave', 'Flag', 'Name', 'Time', 'Net Time', 'Category', 'Cat Pos', 'Gender', 'Gen Pos', 'Club', 'Country']
Successfully extracted 50 data rows.
Creating Pandas DataFrame...
DataFrame created with headers.
Data from page 2 added. Total rows so far: 100
https://results.finishtime.co.za/results.aspx?CId=35&RId=30205&EId=1&dt=0&PageNo=3
Final Headers identified: ['Pos', 'Race No', 'Wave', 'Flag', 'Name', 'Time', 'Net Time', 'Category', 'Cat Pos', 'Gender', 'Gen Pos', 'Club', 'Country']
Succes

In [5]:
all_data  # Display a sample of the data

,Pos,Race No,Wave,Flag,Name,Time,Net Time,Category,Cat Pos,Gender,Gen Pos,Club,Country
0,1,28951,A - Group 1,ZA,Tete DIJANA,05:25:28,05:25:25,Senior,1,Male,1,NEDBANK RUNNING CLUB GN,South Africa
1,2,55335,A - Group 1,NL,Piet WIERSMA,05:25:33,05:25:30,Senior,2,Male,2,NEDBANK INTERNATIONAL,Netherlands
2,3,31470,A - Group 1,XX,Nikolai VOLKOV,05:29:42,05:29:36,Senior,3,Male,3,MAXED ELITE INTERNATIONAL,ANA
3,4,17472,A - Group 1,ZA,Edward MOTHIBI,05:31:41,05:31:38,40-49,1,Male,4,NEDBANK RUNNING CLUB GN,South Africa
4,5,28773,A - Group 1,ZA,Joseph MANYEDI,05:32:09,05:32:04,40-49,2,Male,5,MAXED ELITE KZN,South Africa
...,...,...,...,...,...,...,...,...,...,...,...,...,...
22681,,36241,A - Group 1,US,Devon YANKO,DNS,DNS,40-49,,Female,,,United States of America (the)
22682,,49608,A - Group 1,FR,Pierre-Antoine RUEL,DNS,DNS,Senior,,Male,,SALOMON,France
22683,,54048,A - Group 1,ZA,Janie GRUNDLING,DNS,DNS,40-49,,Female,,PHANTANE AC,South Africa
22684,,57422,A - Group 1,ZA,Siyabonga MAFULEKA,DNS,DNS,Senior,,Male,,SAVAGES AC,South Africa


In [ ]:
# all_data.to_csv('comrades_2025_results.csv', index=False)  # Save the DataFrame to a CSV file